In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
"""
Interface Gradio Complète pour Classification COVID-19
Intégration ResNet50 + VGG16
"""


import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import requests
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from sklearn.metrics import confusion_matrix, classification_report

# Interface
import gradio as gr

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# SECTION 2: CONNEXION À GOOGLE DRIVE


from google.colab import drive
drive.mount('/content/drive')

print(" Google Drive monté avec succès")

# SECTION 3: CHEMINS ET CONFIGURATION

# Chemins vers les modèles
RESNET_MODEL_PATH = "/content/drive/MyDrive/COVID_XRay_Project/final_covid_model.h5"
VGG_MODEL_PATH = "/content/drive/MyDrive/COVID_XRay_Project/best_model.keras"
RESNET_HISTORY_PATH = "/content/drive/MyDrive/COVID_XRay_Project/training_history.pkl"
RESNET_RESULTS_PATH = "/content/drive/MyDrive/COVID_XRay_Project/test_results.pkl"

# Classes
CLASSES = ["COVID", "Normal", "Viral Pneumonia"]

# Couleurs par classe
CLASS_COLORS = {
    "COVID": "#EF476F",      # Rouge
    "Normal": "#06D6A0",     # Vert
    "Viral Pneumonia": "#FFD166"  # Jaune
}

# SECTION 4: CHARGEMENT DES MODÈLES


print("\n Chargement des modèles...")

# Charger ResNet50
try:
    resnet_model = load_model(RESNET_MODEL_PATH, compile=False)
    resnet_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print(" ResNet50 chargé")
except Exception as e:
    print(f" Erreur chargement ResNet50: {e}")
    resnet_model = None

# Charger VGG16
try:
    vgg_model = load_model(VGG_MODEL_PATH, compile=False)
    vgg_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print(" VGG16 chargé")
except Exception as e:
    print(f" Erreur chargement VGG16: {e}")
    vgg_model = None

# Charger historique ResNet
try:
    with open(RESNET_HISTORY_PATH, 'rb') as f:
        resnet_history = pickle.load(f)
    print(" Historique ResNet50 chargé")
except:
    resnet_history = None

# Charger résultats ResNet
try:
    with open(RESNET_RESULTS_PATH, 'rb') as f:
        resnet_results = pickle.load(f)
    print(" Résultats ResNet50 chargés")
except:
    resnet_results = {}

# SECTION 5: FONCTIONS DE PRÉDICTION

def preprocess_image_resnet(img):
    """Prétraiter pour ResNet50"""
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize((200, 200))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = resnet_preprocess(img_array)
    return img_array

def preprocess_image_vgg(img):
    """Prétraiter pour VGG16"""
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize((200, 200))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict_with_resnet(img):
    """Prédiction avec ResNet50"""
    if resnet_model is None:
        return None, None, 0.0

    img_array = preprocess_image_resnet(img)
    predictions = resnet_model.predict(img_array, verbose=0)[0]

    results = {CLASSES[i]: float(predictions[i]) for i in range(len(CLASSES))}
    predicted_class = CLASSES[np.argmax(predictions)]
    confidence = np.max(predictions) * 100

    return results, predicted_class, confidence

def predict_with_vgg(img):
    """Prédiction avec VGG16"""
    if vgg_model is None:
        return None, None, 0.0

    img_array = preprocess_image_vgg(img)
    predictions = vgg_model.predict(img_array, verbose=0)[0]

    results = {CLASSES[i]: float(predictions[i]) for i in range(len(CLASSES))}
    predicted_class = CLASSES[np.argmax(predictions)]
    confidence = np.max(predictions) * 100

    return results, predicted_class, confidence

# SECTION 6: FONCTIONS DE VISUALISATION

def create_comparison_plot(img, resnet_results, resnet_class, resnet_conf,
                          vgg_results, vgg_class, vgg_conf):
    """Créer visualisation comparative des deux modèles"""

    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(2, 3, hspace=0.3, wspace=0.3)

    # Image originale (grande, en haut à gauche)
    ax_img = fig.add_subplot(gs[:, 0])
    ax_img.imshow(img)
    ax_img.set_title('Radiographie Thoracique', fontsize=16, fontweight='bold', pad=20)
    ax_img.axis('off')

    # Prédictions ResNet50
    ax_resnet = fig.add_subplot(gs[0, 1:])
    if resnet_results:
        probs_resnet = [resnet_results[cls] * 100 for cls in CLASSES]
        colors_resnet = [CLASS_COLORS[cls] if cls != resnet_class else '#118AB2'
                        for cls in CLASSES]

        bars_resnet = ax_resnet.barh(CLASSES, probs_resnet, color=colors_resnet,
                                     edgecolor='black', linewidth=2, alpha=0.8)
        ax_resnet.set_xlabel('Probabilité (%)', fontsize=13, fontweight='bold')
        ax_resnet.set_title(f'🏆 ResNet50 - Prédiction: {resnet_class} ({resnet_conf:.2f}%)',
                           fontsize=14, fontweight='bold', color='#118AB2')
        ax_resnet.set_xlim([0, 100])
        ax_resnet.grid(True, alpha=0.3, axis='x')

        for i, (bar, prob) in enumerate(zip(bars_resnet, probs_resnet)):
            ax_resnet.text(prob + 2, bar.get_y() + bar.get_height()/2,
                          f'{prob:.1f}%', va='center', fontsize=11, fontweight='bold')

    # Prédictions VGG16
    ax_vgg = fig.add_subplot(gs[1, 1:])
    if vgg_results:
        probs_vgg = [vgg_results[cls] * 100 for cls in CLASSES]
        colors_vgg = [CLASS_COLORS[cls] if cls != vgg_class else '#073B4C'
                     for cls in CLASSES]

        bars_vgg = ax_vgg.barh(CLASSES, probs_vgg, color=colors_vgg,
                              edgecolor='black', linewidth=2, alpha=0.8)
        ax_vgg.set_xlabel('Probabilité (%)', fontsize=13, fontweight='bold')
        ax_vgg.set_title(f'🥈 VGG16 - Prédiction: {vgg_class} ({vgg_conf:.2f}%)',
                        fontsize=14, fontweight='bold', color='#073B4C')
        ax_vgg.set_xlim([0, 100])
        ax_vgg.grid(True, alpha=0.3, axis='x')

        for i, (bar, prob) in enumerate(zip(bars_vgg, probs_vgg)):
            ax_vgg.text(prob + 2, bar.get_y() + bar.get_height()/2,
                       f'{prob:.1f}%', va='center', fontsize=11, fontweight='bold')

    plt.suptitle('🔬 Comparaison des Modèles de Deep Learning',
                 fontsize=18, fontweight='bold', y=0.98)

    return fig


def create_metrics_table():
    """Créer tableau comparatif des métriques"""
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis('tight')
    ax.axis('off')

    # Données du tableau
    data = [
        ['Modèle', 'Accuracy', 'Précision COVID', 'Recall COVID', 'F1-Score'],
        ['ResNet50', '98.29%', '98.58%', '97.23%', '97.71%'],
        ['VGG16', '97.00%', '92.00%', '88.00%', '95.00%']
    ]

    # Couleurs
    colors = [['#118AB2']*5, ['#06D6A0']*5, ['#FFD166']*5]

    table = ax.table(cellText=data, cellLoc='center', loc='center',
                    cellColours=colors, colWidths=[0.15, 0.15, 0.2, 0.2, 0.15])

    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 3)

    # Style header
    for i in range(5):
        table[(0, i)].set_facecolor('#073B4C')
        table[(0, i)].set_text_props(weight='bold', color='white', size=13)

    # Style rows
    for i in range(1, 3):
        for j in range(5):
            table[(i, j)].set_text_props(weight='bold', size=12)
            table[(i, j)].set_alpha(0.7)

    plt.title('📊 Comparaison des Performances', fontsize=16, fontweight='bold', pad=20)
    return fig

# SECTION 7: FONCTIONS PRINCIPALES DE L'INTERFACE

def predict_and_compare(input_image, model_choice):
    """Fonction principale de prédiction"""
    if input_image is None:
        return None, "⚠️ Veuillez uploader une image", "", ""

    try:
        # Prédictions
        resnet_results, resnet_class, resnet_conf = predict_with_resnet(input_image)
        vgg_results, vgg_class, vgg_conf = predict_with_vgg(input_image)

        # Visualisation
        if model_choice == "Comparaison des deux modèles":
            fig = create_comparison_plot(input_image, resnet_results, resnet_class, resnet_conf,
                                        vgg_results, vgg_class, vgg_conf)
        elif model_choice == "ResNet50 uniquement":
            fig = create_single_model_plot(input_image, resnet_results, resnet_class, resnet_conf, "ResNet50")
        else:  # VGG16
            fig = create_single_model_plot(input_image, vgg_results, vgg_class, vgg_conf, "VGG16")

        # Texte résultats ResNet
        resnet_text = f"""
### 🏆 ResNet50 (Meilleur Modèle)
**Prédiction:** {resnet_class}
**Confiance:** {resnet_conf:.2f}%

**Probabilités:**
- COVID: {resnet_results['COVID']*100:.2f}%
- Normal: {resnet_results['Normal']*100:.2f}%
- Viral Pneumonia: {resnet_results['Viral Pneumonia']*100:.2f}%
"""

        # Texte résultats VGG
        vgg_text = f"""
### 🥈 VGG16
**Prédiction:** {vgg_class}
**Confiance:** {vgg_conf:.2f}%

**Probabilités:**
- COVID: {vgg_results['COVID']*100:.2f}%
- Normal: {vgg_results['Normal']*100:.2f}%
- Viral Pneumonia: {vgg_results['Viral Pneumonia']*100:.2f}%
"""

        # Recommandation
        final_class = resnet_class  # On privilégie ResNet (meilleur modèle)
        recommendation = get_recommendation(final_class)

        return fig, resnet_text, vgg_text, recommendation

    except Exception as e:
        return None, f"❌ Erreur: {str(e)}", "", ""

def create_single_model_plot(img, results, pred_class, confidence, model_name):
    """Créer plot pour un seul modèle"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Image
    ax1.imshow(img)
    ax1.set_title(f'Radiographie\nPrédiction: {pred_class}',
                 fontsize=14, fontweight='bold')
    ax1.axis('off')

    # Barres
    probs = [results[cls] * 100 for cls in CLASSES]
    colors = [CLASS_COLORS[cls] if cls != pred_class else '#118AB2' for cls in CLASSES]

    bars = ax2.barh(CLASSES, probs, color=colors, edgecolor='black', linewidth=2)
    ax2.set_xlabel('Probabilité (%)', fontsize=12, fontweight='bold')
    ax2.set_title(f'{model_name}\nConfiance: {confidence:.2f}%',
                 fontsize=14, fontweight='bold')
    ax2.set_xlim([0, 100])
    ax2.grid(True, alpha=0.3, axis='x')

    for bar, prob in zip(bars, probs):
        ax2.text(prob + 2, bar.get_y() + bar.get_height()/2,
                f'{prob:.1f}%', va='center', fontsize=11, fontweight='bold')

    plt.tight_layout()
    return fig

def get_recommendation(predicted_class):
    """Obtenir recommandation médicale"""
    recommendations = {
        'COVID': """
### ⚠️ Recommandations - COVID-19 Détecté

**Actions immédiates:**
- ✅ Consulter un médecin rapidement
- ✅ Effectuer un test PCR de confirmation
- ✅ S'isoler immédiatement
- ✅ Surveiller les symptômes

**Important:** Cette prédiction doit être confirmée par un professionnel de santé.
""",
        'Viral Pneumonia': """
### ⚠️ Recommandations - Pneumonie Virale Détectée

**Actions recommandées:**
- ✅ Consultation médicale recommandée
- ✅ Traitement antiviral possible
- ✅ Surveillance de la saturation en oxygène
- ✅ Repos et hydratation

**Important:** Nécessite une évaluation clinique complète.
""",
        'Normal': """
### ✅ Recommandations - Radiographie Normale

**Observations:**
- Aucune anomalie détectée
- Poumons apparemment sains

**Si symptômes persistent:**
- Consulter un médecin
- Des tests complémentaires peuvent être nécessaires

**Important:** Un résultat normal ne remplace pas un examen médical complet.
"""
    }

    return recommendations.get(predicted_class, "Aucune recommandation disponible")

# ============================================================================
# SECTION 8: INTERFACE GRADIO
# ============================================================================

# CSS personnalisé
custom_css = """
.gradio-container {
    max-width: 1400px !important;
    margin: auto !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

.header-title {
    text-align: center;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 30px;
    border-radius: 15px;
    margin-bottom: 20px;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}

.metric-card {
    background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
    color: white;
    padding: 20px;
    border-radius: 10px;
    margin: 10px 0;
    text-align: center;
    font-weight: bold;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}

.warning-box {
    background-color: #fff3cd;
    border-left: 5px solid #ffc107;
    padding: 15px;
    margin: 15px 0;
    border-radius: 5px;
}

.success-box {
    background-color: #d4edda;
    border-left: 5px solid #28a745;
    padding: 15px;
    margin: 15px 0;
    border-radius: 5px;
}

footer {
    text-align: center;
    padding: 20px;
    background-color: #f8f9fa;
    margin-top: 30px;
    border-radius: 10px;
}
"""

# Créer l'interface
with gr.Blocks(css=custom_css, theme=gr.themes.Soft(), title="COVID-19 X-Ray Classification") as demo:

    # En-tête
    gr.HTML("""
    <div class="header-title">
        <h1 style="margin: 0; font-size: 32px;">🔬 Détection Automatique de COVID-19 par Radiographies Thoraciques</h1>
        <p style="margin: 10px 0 0 0; font-size: 18px;">
            Comparaison ResNet50 vs VGG16 - Master IA & IoT - Université Ibn Tofail
        </p>
        <p style="margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;">
            Par: Hafsa Kaalal & Mariyame Tazi | Encadrant: Pr. Mohtaram Nourredine
        </p>
    </div>
    """)

    with gr.Tabs() as tabs:

        # ============ ONGLET 1: PRÉDICTION ============
        with gr.TabItem("🔍 Prédiction d'Images", id=0):
            with gr.Row():
                with gr.Column(scale=1):
                    gr.Markdown("### 📤 Upload Radiographie")
                    image_input = gr.Image(type="pil", label="Radiographie thoracique")

                    model_choice = gr.Radio(
                        choices=["Comparaison des deux modèles", "ResNet50 uniquement", "VGG16 uniquement"],
                        value="Comparaison des deux modèles",
                        label="Mode d'affichage"
                    )

                    predict_btn = gr.Button("🚀 Analyser", variant="primary", size="lg")
                    clear_btn = gr.Button("🗑️ Effacer", variant="secondary")

                    gr.Markdown("### 🎯 Métriques des Modèles")
                    gr.HTML("""
                    <div class="metric-card">
                        <h3>ResNet50</h3>
                        <p>Accuracy: 98.29% | F1: 97.71%</p>
                    </div>
                    <div class="metric-card">
                        <h3>VGG16</h3>
                        <p>Accuracy: 97.00% | F1: 95.00%</p>
                    </div>
                    """)

                with gr.Column(scale=2):
                    gr.Markdown("### 📊 Résultats de l'Analyse")
                    plot_output = gr.Plot(label="Visualisation")

                    with gr.Row():
                        with gr.Column():
                            resnet_output = gr.Markdown(label="ResNet50")
                        with gr.Column():
                            vgg_output = gr.Markdown(label="VGG16")

                    recommendation_output = gr.Markdown(label="Recommandations")

            # Events
            predict_btn.click(
                predict_and_compare,
                inputs=[image_input, model_choice],
                outputs=[plot_output, resnet_output, vgg_output, recommendation_output]
            )

            clear_btn.click(
                lambda: [None, None, "", "", ""],
                outputs=[image_input, plot_output, resnet_output, vgg_output, recommendation_output]
            )



        # ============ ONGLET 3: À PROPOS ============
        with gr.TabItem("ℹ️ À Propos", id=2):
            gr.Markdown("""
            ## 🎓 Projet de Classification COVID-19

            ### 📋 Objectif
            Ce système utilise des techniques de **Deep Learning** pour classifier automatiquement
            les radiographies thoraciques en trois catégories:
            - 🦠 **COVID-19**
            - ✅ **Normal**
            - 🫁 **Pneumonie Virale**

            ### 🏆 Performances

            | Modèle | Accuracy | Précision COVID | Recall COVID | F1-Score |
            |--------|----------|-----------------|--------------|----------|
            | **ResNet50** | **98.29%** | **98.58%** | **97.23%** | **97.71%** |
            | **VGG16** | 97.00% | 92.00% | 88.00% | 95.00% |

            ### 🔬 Spécifications Techniques
            - **Dataset:** 16,395 radiographies thoraciques
            - **Architecture ResNet50:** Transfer Learning avec fine-tuning
            - **Architecture VGG16:** Transfer Learning
            - **Framework:** TensorFlow/Keras
            - **Interface:** Gradio
            - **Validation:** K-Fold Cross-Validation (K=5)

            ### ⚠️ Avertissements Importants

            <div class="warning-box">
            <strong>⚠️ ATTENTION</strong><br>
            Ce système est un <strong>outil d'aide à la décision</strong> et ne remplace en aucun cas:
            <ul>
                <li>Un diagnostic médical professionnel</li>
                <li>L'expertise d'un radiologiste qualifié</li>
                <li>Des tests de confirmation (PCR, etc.)</li>
            </ul>
            Les prédictions doivent <strong>toujours être validées</strong> par un professionnel de santé.
            </div>

            ### 👥 Équipe
            - **Étudiantes:** Hafsa Kaalal, Mariyame Tazi
            - **Encadrant:** Pr. Mohtaram Nourredine
            - **Formation:** Master d'Excellence en Intelligence Artificielle et IoT
            - **Institution:** Université Ibn Tofail
            - **Année:** 2025-2026

            ### 📚 Références
            - Article ACM: "A Deep Learning Approach for COVID-19 & Viral Pneumonia Screening with X-ray Images" (2021)
            - He et al., "Deep Residual Learning for Image Recognition" (2016)
            - Simonyan & Zisserman, "Very Deep Convolutional Networks" (2015)

            ### 📧 Contact
            Pour toute question ou collaboration: [email protégé]
            """)

    # Pied de page
    gr.HTML("""
    <footer>
        <p style="font-size: 16px; margin: 10px 0;">
            <strong>🏥 Application de Recherche Médicale</strong>
        </p>
        <p style="color: #dc3545; font-weight: bold; margin: 10px 0;">
            ⚠️ À utiliser uniquement à des fins de recherche et d'éducation
        </p>
        <p style="color: #6c757d; font-size: 14px; margin: 10px 0;">
            Développé avec ❤️ en utilisant TensorFlow, Keras et Gradio<br>
            © 2026 - Université Ibn Tofail - Master IA & IoT
        </p>
    </footer>
    """)

# ============================================================================
# SECTION 9: LANCEMENT
# ============================================================================

print("\n" + "="*70)
print("🚀 LANCEMENT DE L'APPLICATION")
print("="*70)

# Lancer l'interface
demo.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860,
    show_error=True
)

print("\n Application lancée avec succès!")
print(" Utilisez le lien 'share' pour accéder à l'interface")
print("="*70)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Google Drive monté avec succès

 Chargement des modèles...
 ResNet50 chargé
 VGG16 chargé
 Historique ResNet50 chargé
 Résultats ResNet50 chargés


/tmp/ipython-input-1884904167.py:434: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft(), title="COVID-19 X-Ray Classification") as demo:
/tmp/ipython-input-1884904167.py:434: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft(), title="COVID-19 X-Ray Classification") as demo:



🚀 LANCEMENT DE L'APPLICATION
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://30045c24a729902504.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://30045c24a729902504.gradio.live

 Application lancée avec succès!
 Utilisez le lien 'share' pour accéder à l'interface


/usr/local/lib/python3.12/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 127942 (\N{TROPHY}) missing from font(s) Liberation Sans.
  func(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 129352 (\N{SECOND PLACE MEDAL}) missing from font(s) Liberation Sans.
  func(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/IPython/core/events.py:89: UserWarning: Glyph 128300 (\N{MICROSCOPE}) missing from font(s) Liberation Sans.
  func(*args, **kwargs)
